In [86]:
from sympy import symbols, solve, Array, tensorproduct, Matrix, init_printing, pprint, zeros, MatMul, Eq, latex, factor


In [87]:
# A is a n x n complex matrix
n = 2

# makes a 2d array, in our case:
# a list containing n empty lists

matrix = [[] for _ in range(n*n)]
for i in range(n*n):
    for j in range(n*n):
        matrix[i].append(symbols(f"a{i+1}{j+1}"))
print(matrix)

[[a11, a12, a13, a14], [a21, a22, a23, a24], [a31, a32, a33, a34], [a41, a42, a43, a44]]


In [88]:
A = Matrix(matrix)
# included in iPython 6.0 display will choose appropriate representation (text/LaTeX/png)
# you can think of iPython as the interpreter that runs in jupyter notebooks
print(A.shape)
display(A)

(4, 4)


⎡a₁₁  a₁₂  a₁₃  a₁₄⎤
⎢                  ⎥
⎢a₂₁  a₂₂  a₂₃  a₂₄⎥
⎢                  ⎥
⎢a₃₁  a₃₂  a₃₃  a₃₄⎥
⎢                  ⎥
⎣a₄₁  a₄₂  a₄₃  a₄₄⎦

In [89]:
# These are our elementary 2x2 matrices unrolled into a column vector
e11 = Matrix([[1], [0], [0], [0]])
e12 = Matrix([[0], [1], [0], [0]])
e21 = Matrix([[0], [0], [1], [0]])
e22 = Matrix([[0], [0], [0], [1]])
display(e11)


⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [90]:
# this generates a map square -> sqrt for all the perfect squares in [1,81]
squares = {n**2:n for n in range(1,10)}

def rollMatrix(matrix):
    # rows and cols
    r,c = matrix.shape
    # if a column vector with n^2 entries
    # we can roll into n x n matrix
    if r not in squares or c != 1:
        print("ex")
        raise Exception("Matrix cannot be unrolled, make sure matrix \
            is square and has atleast 1 entry")
    # the sqrt of r (# of rows) is n for our n x n matrix output
    n = squares[r]
    # n x n matrix of zeros
    new_matrix = [[0] * n for _ in range(n)]
    
    for i in range(n):
        for j in range(n):
            # remember when indexing 2d arrays, first num is row, second is col
            # also remember python is zero indexed while matlab is 1 indexed
            new_matrix[i][j] = matrix[i*n + j]
            
    return Matrix(new_matrix)
# example!
rollMatrix(Matrix([[0],[1],[2],[3],[4],[5],[6],[7],[8]]))


⎡0  1  2⎤
⎢       ⎥
⎢3  4  5⎥
⎢       ⎥
⎣6  7  8⎦

In [91]:
def unrollMatrix(matrix):
    # rows and cols
    r,c = matrix.shape
    # if a n x n matrix and 
    # we can roll into n x n matrix
    if r != c or r <= 0:
        print("ex")
        raise Exception("Matrix cannot be unrolled, make sure matrix \
            has a square number of entries and 1 column")
    
    # the sqrt of r (# of rows) is n for our n x n matrix output
    n = r
    # if this looks really confusing its called list comprehension
    # and is super powerful, but can get ugly
    # n x n matrix of zeros
    
    # this works because in sympy matrices
    # can be indexed by one number
    # i think this shouldnt work but i did not make sympy
    new_matrix = [ matrix[i] for i in range(n*n)]
    
    return Matrix(new_matrix)
# example
unrollMatrix(Matrix([[0,1,2],[3,4,5],[6,7,8]]))

⎡0⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎢2⎥
⎢ ⎥
⎢3⎥
⎢ ⎥
⎢4⎥
⎢ ⎥
⎢5⎥
⎢ ⎥
⎢6⎥
⎢ ⎥
⎢7⎥
⎢ ⎥
⎣8⎦

In [94]:
# this will make e[1][1] = e11
e = [
     [None, None, None],
     [None, e11, e12],
     [None, e21, e22]
    ]
print(n)
s = [[zeros(n,n)]*3 for _ in range(3)] 
for i in range(1,1+n):
    for j in range(1,1+n):
        for k in range(1,1+n):
            print(i,j,k)
            s[i][j] += 2*(rollMatrix(A*e[i][k]) * rollMatrix(A*e[k][j]))
display(s[1][1], s[1][2], s[2][1], s[2][2])

2
1 1 1
1 1 2
1 2 1
1 2 2
2 1 1
2 1 2
2 2 1
2 2 2


⎡     2                                ⎤
⎢2⋅a₁₁  + 2⋅a₂₂⋅a₃₃          0         ⎥
⎢                                      ⎥
⎢                                     2⎥
⎣        0           2⋅a₂₃⋅a₃₂ + 2⋅a₄₁ ⎦

⎡          0            2⋅a₁₁⋅a₂₂ + 2⋅a₂₂⋅a₄₄⎤
⎢                                            ⎥
⎣2⋅a₁₄⋅a₃₂ + 2⋅a₃₂⋅a₄₁            0          ⎦

⎡          0            2⋅a₁₄⋅a₂₃ + 2⋅a₂₃⋅a₄₁⎤
⎢                                            ⎥
⎣2⋅a₁₁⋅a₃₃ + 2⋅a₃₃⋅a₄₄            0          ⎦

⎡     2                                ⎤
⎢2⋅a₁₄  + 2⋅a₂₃⋅a₃₂          0         ⎥
⎢                                      ⎥
⎢                                     2⎥
⎣        0           2⋅a₂₂⋅a₃₃ + 2⋅a₄₄ ⎦

In [101]:
solve(rollMatrix(-4*A*e[1][1]) + s[1][1], A[0])
equations = []
A[1] = A[2] = A[4] = A[7] = A[8] = A[11] = A[13] = A[14] = 0
A[
for i in range(1,3):
    for j in range(1,3):
        for k in range(4):
            equations.append([rollMatrix(A*e[i][j])[k], s[i][j][k]/4])

symbols = []
# for i in range(n*n*n*n):
#     symbols.append(A[i])
# This takes too long to run
# display(solve(equations,symbols))
            
display(equations)
display(A)
display(rollMatrix(4*A*e[1][1]))
(s[1][1])
(rollMatrix(4*A*e[1][1]))
equality = Eq(rollMatrix(4*A*e[1][1]), s[1][1])
display(equality)
# solve(equality, A[0])

for a, eq in equations:
    equal = Eq(2*a,2*factor(eq))
#     display(factor(eq))
    print("$", latex(equal), "$\\\\")
solve(equations[0][0]- equations[0][1], A[0])

⎡a₁₁   0    0   a₁₄⎤
⎢                  ⎥
⎢ 0   a₂₂  a₂₃   0 ⎥
⎢                  ⎥
⎢ 0   a₃₂  a₃₃   0 ⎥
⎢                  ⎥
⎣a₄₁   0    0   a₄₄⎦

⎡4⋅a₁₁    0  ⎤
⎢            ⎥
⎣  0    4⋅a₄₁⎦

                 ⎡     2                                ⎤
⎡4⋅a₁₁    0  ⎤   ⎢2⋅a₁₁  + 2⋅a₂₂⋅a₃₃          0         ⎥
⎢            ⎥ = ⎢                                      ⎥
⎣  0    4⋅a₄₁⎦   ⎢                                     2⎥
                 ⎣        0           2⋅a₂₃⋅a₃₂ + 2⋅a₄₁ ⎦

$ 2 a_{11} = a_{11}^{2} + a_{22} a_{33} $\\
$ \text{True} $\\
$ \text{True} $\\
$ 2 a_{41} = a_{23} a_{32} + a_{41}^{2} $\\
$ \text{True} $\\
$ 2 a_{22} = a_{22} \left(a_{11} + a_{44}\right) $\\
$ 2 a_{32} = a_{32} \left(a_{14} + a_{41}\right) $\\
$ \text{True} $\\
$ \text{True} $\\
$ 2 a_{23} = a_{23} \left(a_{14} + a_{41}\right) $\\
$ 2 a_{33} = a_{33} \left(a_{11} + a_{44}\right) $\\
$ \text{True} $\\
$ 2 a_{14} = a_{14}^{2} + a_{23} a_{32} $\\
$ \text{True} $\\
$ \text{True} $\\
$ 2 a_{44} = a_{22} a_{33} + a_{44}^{2} $\\
